In [237]:
#
# scraping 5 essentials website, 2018 public report
#

import requests
import re
from bs4 import BeautifulSoup
import json
import pandas
from pandas import json_normalize
import re
import numpy as np

##### Set these constants before running #####
reportyr = 2018

# source: Chicago data portal...update with different years or filter for specific school subsets if desired

schls = pandas.read_csv("data/Chicago_Public_Schools_-_School_Profile_Information_SY1819.csv")
schl_ids = schls["School_ID"]

outpath = "/Users/mercedeswentworth-nice/Documents/Spring Quarter 2020/PBPL 26303/fiveE2018.csv"

##### Funcs #####
def get5Edescription(s):
    s = str(s)
    start = s.find("<br>") + len("<br>")
    end = s.find(".")
    substring = s[start:end]
    return substring

def enumerateYear(x, reportyr):
    if type(x) == list:
        for i in range(0,len(x)):
            if type(x[i]) == list:
                x[i].insert(0, reportyr-i)
    return x

def getSchoolData(cpsid, year):
    # grab page
    page = requests.get("https://www.5-essentials.org/cps/5e/" + str(reportyr) + "/s/"+ str(cpsid) + "/measures/")

    soup = BeautifulSoup(page.content, 'html.parser')

    if soup.find("title").get_text() == "Page Not Found":
        return pandas.DataFrame([])
    else: 
        # get data, in json object form
        data = soup.find(id="data")

        # get title node and extract school name
        title = soup.find("title").text
        schlname = " ".join(filter(lambda x: x != "", re.split("\n| |\t|5Essentials|Report", title)))

        # extract measures from object and convert to pandas data frame
        jsonObj = json.loads(data['data-data'])
        measures = jsonObj["measures"]
        df = json_normalize(measures)

        # clean data frame
        df.insert(0, "schlname", [schlname]*len(df.index))
        df.insert(5, "description", df["text.overview.__html"].apply(get5Edescription))
        df = df.filter(items = ["schlname","name","level", "stem", "description", "subject", "data.all"])
        df = df.applymap(lambda x : enumerateYear(x, 2018))
        df = df.explode("data.all")
        df["year"] = df["data.all"].apply(lambda x: x[0] if type(x) == list else x)
        df["data.all"] = df["data.all"].apply(lambda x: x[1] if type(x) == list else x)
        df["School_ID"] = cpsid
        return df

##### Scrape #####
schl_data = []
for i in schl_ids:
    print(i)
    schl_data.append(getSchoolData(i, reportyr))    

pandas.concat(schl_data).to_csv(outpath)



610191
609966
400069
400173
400057
610153
610117
610092
610237
609908
609945
610532
609704
609954
610152
609972
610299
609734
400106
610097
610203
610108
400026
610047
610308
610135
610102
610276
610212
609976
400080
609942
400062
609902
400126
610590
609928
609750
610087
610034
610185
610036
400028
610369
610057
400085
610594
610091
610242
610175
400050
610252
609871
610105
609983
609727
400157
610188
609839
400131
609806
609676
609803
400089
609698
610209
400071
400030
400136
610202
610084
609729
610176
610043
610300
400092
400180
400170
400075
400055
400179
400161
400164
609880
609964
610319
609887
400135
610279
610046
609760
610316
609845
400181
610136
609929
610238
609903
609991
400036
610323
400013
610056
609910
610182
609867
610139
610249
610515
609708
400064
610293
609768
610225
609789
400046
610229
610063
610078
610100
609828
400009
610089
609958
609712
609855
400115
610174
400178
400047
610026
609821
610053
610106
610048
609865
609869
609745
610390
609950
609852
400052
610313